In [33]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs
import re

## STEP.1 
### 自結損益爬蟲
### issue1:2882公布月自結非季自結，2330和1301皆公布季自結 (sloved)
### issue2:自結EPS 不在自結損益公告項目內，目前無法解決

In [90]:
def get_self_acc_sales(stk, year):
    # 公開資訊觀測站 自結損益公告-季申報
    url = 'https://mops.twse.com.tw/mops/web/ajax_t138sb02'
        
    # 先從季公告開始，若季公告沒資料則去爬月公告
    freq = 'Q'
    soup = request_url(url, stk, year, freq)
    if len(soup.find_all('font',{'color':'red'})) > 0:
        freq = 'M'
        soup = request_url(url, stk, year, freq)
        # print(soup)
    self_acc_sales = soup.find_all('td',{'align':'right', 'class':'odd', 'width':''})
    # 最新一季資料
    if freq == 'Q':
        latest = extract_number(self_acc_sales[-1])
    else:
        latest = extract_number(self_acc_sales[-3:])

    code_name = [i for i in soup.find_all('h3') if '公司名稱' in str(i)]
    # 取得 股票名稱
    # \n表示換行
    temp = str(code_name[0]).split('\r\n')[1]
    name = temp.split(' ')[0]
    #
    data = {
        '代碼':stk,
        '股名':name,
        '自結上一季獲利':latest
    }
    result = pd.DataFrame([data])
    return result

def get_payload(stk, year, freq):
        payload = {
                'encodeURIComponent':'1',
                'run':'Y',
                'step': '1',
                'CK2': '3',
                'BK1': '2',
                'TYPEK':'sii', 
                'YEAR':year, 
                'COMP': stk,
                'firstin': 'true'
        }
        if freq == 'M':
            payload['CK2'] = '1'
        return payload

def request_url(url, stk, year, freq):
    payload = get_payload(stk, year, freq)
    response = rq.get(url, params=payload)
    soup = bs(response.content, 'html.parser') 
    return soup

def extract_number(lst):
    def extract_num(item):
        item = str(item)
        number = re.search(r'\d[\d,]*\d', item).group()
        number_str = number.replace(',','').replace('-','')
        number = int(number_str)
        if '-' in item:
            number = (-1)*number
        return number
    numbers = list(map(extract_num, lst))
    return sum(numbers)

In [91]:
data = pd.DataFrame()
stks = ['2882','2330','1301']
for stk in stks:
    temp = get_self_acc_sales(stk,'111')
    data = pd.concat([data, temp])
data = data.reset_index(drop=True)
data

,代碼,股名,自結上一季獲利
0,2882,國泰金,-20543450
0,2330,台積電,334669874
0,1301,台塑,-7305379


## STEP. 2
### 資料匯進 excel

In [92]:
import openpyxl

In [98]:
len(data)

3

In [100]:
# 讀取 excel
workbook = openpyxl.load_workbook('試題.xlsx')
# 選取 試題一 的工作表
sheet = workbook.worksheets[2]
fill_columns = ['A', 'B','C']
for col in fill_columns:
    for index in range(len(data)):
        sheet[col+str(index+4)] = data.iloc[index, fill_columns.index(col)]

In [104]:
# 儲存檔案
workbook.save('test.xlsx')